In [3]:
import open3d as o3d
import numpy as np
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
def load_point_clouds(voxel_size=0.0):
    pcds = []
    paths = [
        # "C:/Users/nguye/Downloads/images/test.ply",
        # "C:/Users/nguye/Downloads/image2/test2.ply",
        "corner2_voxel.ply",
        "corner1_voxel.ply",
    ]
    for path in paths:
        pcd = o3d.io.read_point_cloud(path)
        pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
        pcds.append(pcd_down)
    return pcds

In [5]:
voxel_size = 0.001
pcds_down = load_point_clouds(voxel_size)
o3d.visualization.draw_geometries(pcds_down,
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])
for i in range(len(pcds_down)):
    pcds_down[i].estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2, max_nn=30))

In [6]:
def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.pipelines.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.pipelines.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(
                        np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(source_id,
                                                             target_id,
                                                             transformation_icp,
                                                             information_icp,
                                                             uncertain=True))
    return pose_graph

In [7]:
print("Full registration ...")
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    pose_graph = full_registration(pcds_down,
                                   max_correspondence_distance_coarse,
                                   max_correspondence_distance_fine)

Full registration ...
Apply point-to-plane ICP
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
[Open3D DEBUG] ICP Iteration #0: Fitness 0.0000, RMSE 0.0000
Build o3d.pipelines.registration.PoseGraph


In [10]:
print("Optimizing PoseGraph ...")
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.05,
    reference_node=0)
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    o3d.pipelines.registration.global_optimization(
        pose_graph,
        o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
        option)

Optimizing PoseGraph ...
[Open3D DEBUG] Validating PoseGraph - finished.
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 0.000000
[Open3D DEBUG] [Initial     ] residual : 0.000000e+00, lambda : 0.000000e+00
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] [GlobalOptimizationLM] Optimizing PoseGraph having 2 nodes and 1 edges.
[Open3D DEBUG] Line process weight : 0.000000
[Open3D DEBUG] [Initial     ] residual : 0.000000e+00, lambda : 0.000000e+00
[Open3D DEBUG] Maximum coefficient of right term < 1.000000e-06
[Open3D DEBUG] CompensateReferencePoseGraphNode : reference : 0


In [ ]:
pcds = load_point_clouds(voxel_size)
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
pcd_combined_down = pcd_combined.voxel_down_sample(voxel_size=voxel_size)
o3d.io.write_point_cloud("multiway_registration.pcd", pcd_combined_down)
o3d.visualization.draw_geometries([pcd_combined_down],
                                  zoom=0.3412,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

: 

In [61]:
# import numpy as np
# import open3d as o3d

# def voxelize_point_cloud(ply_file_path, voxel_size):
#     """
#     Voxelize a point cloud from a PLY file:
#     1. Load the point cloud
#     2. Divide space into voxels of specified size
#     3. For each voxel containing points, keep only one point and move it to the upper right corner
    
#     Args:
#         ply_file_path (str): Path to the PLY file
#         voxel_size (float): Size of each voxel side
    
#     Returns:
#         open3d.geometry.PointCloud: Voxelized point cloud
#     """
#     # Load the point cloud
#     pcd = o3d.io.read_point_cloud(ply_file_path)
#     points = np.asarray(pcd.points)
    
#     if len(points) == 0:
#         print("Empty point cloud")
#         return pcd
    
#     # Calculate min and max coordinates of the point cloud
#     min_bound = np.min(points, axis=0)
#     max_bound = np.max(points, axis=0)
    
#     # Calculate dimensions of the voxel grid
#     num_voxels = np.ceil((max_bound - min_bound) / voxel_size).astype(int)
    
#     # Dictionary to store voxel indices and their corresponding points
#     voxel_dict = {}
    
#     # Assign points to voxels
#     for i, point in enumerate(points):
#         # Calculate voxel indices for the point
#         voxel_index = tuple(np.floor((point - min_bound) / voxel_size).astype(int))
        
#         # Store the point index in the voxel dictionary
#         if voxel_index not in voxel_dict:
#             voxel_dict[voxel_index] = i
    
#     # Create a new point cloud with only one point per voxel, positioned at the corner
#     new_points = []
    
#     for voxel_index, point_index in voxel_dict.items():
#         # Calculate the upper right corner of the voxel
#         voxel_corner = min_bound + np.array(voxel_index) * voxel_size + voxel_size
        
#         # Add the corner point to the new points list
#         new_points.append(voxel_corner)
    
#     # Create a new point cloud with the processed points
#     new_pcd = o3d.geometry.PointCloud()
#     new_pcd.points = o3d.utility.Vector3dVector(np.array(new_points))
    
#     # If the original point cloud has colors, you might want to handle them here
#     # This simple implementation doesn't preserve colors
    
#     print(f"Original point cloud: {len(points)} points")
#     print(f"Voxelized point cloud: {len(new_points)} points")
    
#     return new_pcd

# # Example usage
# if __name__ == "__main__":
#     # Example parameters
#     ply_file = "C:/Users/nguye/Downloads/images/test.ply"
#     voxel_size = 1 # 5cm voxel size
    
#     # Voxelize the point cloud
#     voxelized_pcd = voxelize_point_cloud(ply_file, voxel_size)
    
#     # Visualize the result
#     o3d.visualization.draw_geometries([voxelized_pcd])
    
#     # Optionally save the result
#     o3d.io.write_point_cloud("voxelized_pointcloud.ply", voxelized_pcd)

In [70]:
import open3d as o3d
import numpy as np
from scipy.spatial.transform import Rotation as R

# Camera data (Modify with your real values)
camera_data = {
    "name": "frame000001.png",
    "x": 16.07672148106882,
    "y": -13.36208053348467,
    "alt": 21.43087990863757,
    "heading": -43.76176907447967,  # Yaw
    "pitch": 57.05063527217181,     # Pitch
    "roll": -1.699805914025459,     # Roll
    "f": 29.80963545052193,         # Focal length
    "px": -8.607111067081718e-4,    # Principal point X
    "py": 6.034445625206664e-3,     # Principal point Y
    "k1": 0.1463257180718982,       # Radial distortion coefficients
    "k2": -0.7163134720697377,
    "k3": 1.231617563039704,
    "k4": 0,
    "t1": 0,
    "t2": 0
}

# Load the point cloud
point_cloud_path = "C:/Users/nguye/Downloads/image2/test2.ply"  # Replace with your point cloud file
pcd = o3d.io.read_point_cloud(point_cloud_path)

def get_camera_extrinsics(camera_data):
    """
    Computes the camera extrinsic matrix from position and rotation.
    """
    # Convert heading (yaw), pitch, roll from degrees to radians
    yaw = np.radians(camera_data["heading"])
    pitch = np.radians(camera_data["pitch"])
    roll = np.radians(camera_data["roll"])

    # Create a rotation matrix from Euler angles
    rotation = R.from_euler('zyx', [yaw, pitch, roll], degrees=False).as_matrix()

    # Camera translation (world position)
    translation = np.array([0,0,0])

    # Build the 4x4 extrinsic transformation matrix
    # extrinsic_matrix = np.eye(4)
    # extrinsic_matrix[:3, :3] = rotation
    extrinsic_matrix[:3, 3] = translation

    return extrinsic_matrix

# Compute camera extrinsics
extrinsic_matrix = get_camera_extrinsics(camera_data)

# Transform point cloud to camera space
pcd.transform(np.linalg.inv(extrinsic_matrix))

# Visualize the transformed point cloud
o3d.visualization.draw_geometries([pcd], window_name="Camera View Point Cloud")


In [ ]:
import numpy as np
import open3d as o3d
import math

def parse_camera_params(camera_line):
    """Parse a line of camera parameters."""
    parts = camera_line.strip().split(',')
    # Skip header line if present
    if parts[0] == "#name":
        return None
        
    camera_params = {
        'name': parts[0],
        'x': float(parts[1]),
        'y': float(parts[2]),
        'alt': float(parts[3]),  # altitude/z
        'heading': float(parts[4]),  # in degrees
        'pitch': float(parts[5]),    # in degrees
        'roll': float(parts[6]),     # in degrees
        'f': float(parts[7]),        # focal length
        'px': float(parts[8]),       # principal point x
        'py': float(parts[9]),       # principal point y
        'k1': float(parts[10]),      # radial distortion coefficient
        'k2': float(parts[11]),      # radial distortion coefficient
        'k3': float(parts[12]),      # radial distortion coefficient
        'k4': float(parts[13]),      # radial distortion coefficient
        't1': float(parts[14]),      # tangential distortion coefficient
        't2': float(parts[15])       # tangential distortion coefficient
    }
    return camera_params

def calculate_view_parameters(camera_params):
    """
    Calculate Open3D visualization parameters from camera parameters.
    
    Args:
        camera_params (dict): Camera parameters
        
    Returns:
        dict: Open3D visualization parameters (zoom, front, lookat, up)
    """
    # Extract camera position
    camera_pos = np.array([camera_params['x'], camera_params['y'], camera_params['alt']])
    
    # Convert angles from degrees to radians
    heading_rad = math.radians(camera_params['heading'])
    pitch_rad = math.radians(camera_params['pitch'])
    roll_rad = math.radians(camera_params['roll'])
    
    # Create rotation matrices
    Rx = np.array([
        [1, 0, 0],
        [0, math.cos(roll_rad), -math.sin(roll_rad)],
        [0, math.sin(roll_rad), math.cos(roll_rad)]
    ])
    
    Ry = np.array([
        [math.cos(pitch_rad), 0, math.sin(pitch_rad)],
        [0, 1, 0],
        [-math.sin(pitch_rad), 0, math.cos(pitch_rad)]
    ])
    
    Rz = np.array([
        [math.cos(heading_rad), -math.sin(heading_rad), 0],
        [math.sin(heading_rad), math.cos(heading_rad), 0],
        [0, 0, 1]
    ])
    
    # Compute rotation matrix (order: Z, Y, X)
    R = Rz @ Ry @ Rx
    
    # Calculate front vector (camera direction)
    # In camera coordinates, front is typically along the negative z-axis
    front_camera = np.array([0, 0, -1])
    front = R @ front_camera
    front = front / np.linalg.norm(front)  # Normalize
    
    # Calculate up vector (camera up direction)
    # In camera coordinates, up is typically along the positive y-axis
    up_camera = np.array([0, 1, 0])
    up = R @ up_camera
    up = up / np.linalg.norm(up)  # Normalize
    
    # Calculate lookat point (point the camera is looking at)
    # We'll use the camera direction and a distance based on focal length
    # Typically, lookat = camera_pos + front * some_distance
    lookat_distance = camera_params['f'] * 0.5  # Adjust this multiplier as needed
    lookat = camera_pos + front * lookat_distance
    
    # Calculate zoom based on focal length
    # This is a heuristic and may need adjustment
    zoom = 0.5 + (camera_params['f'] / 50.0)  # Adjust this formula as needed
    
    return {
        'zoom': zoom,
        'front': front.tolist(),
        'lookat': lookat.tolist(),
        'up': up.tolist()
    }

def create_camera_visualization(camera_params, size=0.5):
    """Create a visualization of the camera as a coordinate frame."""
    camera_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=size)
    
    # Convert angles from degrees to radians
    heading_rad = math.radians(camera_params['heading'])
    pitch_rad = math.radians(camera_params['pitch'])
    roll_rad = math.radians(camera_params['roll'])
    
    # Create rotation matrix from Euler angles
    Rx = np.array([
        [1, 0, 0],
        [0, math.cos(roll_rad), -math.sin(roll_rad)],
        [0, math.sin(roll_rad), math.cos(roll_rad)]
    ])
    
    Ry = np.array([
        [math.cos(pitch_rad), 0, math.sin(pitch_rad)],
        [0, 1, 0],
        [-math.sin(pitch_rad), 0, math.cos(pitch_rad)]
    ])
    
    Rz = np.array([
        [math.cos(heading_rad), -math.sin(heading_rad), 0],
        [math.sin(heading_rad), math.cos(heading_rad), 0],
        [0, 0, 1]
    ])
    
    R = Rz @ Ry @ Rx
    
    # Create transformation matrix
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = [camera_params['x'], camera_params['y'], camera_params['alt']]
    
    # Apply transformation
    camera_frame.transform(T)
    
    return camera_frame

def render_point_cloud_from_camera(ply_file_path, camera_params):
    """
    Render a point cloud from a specific camera viewpoint.
    
    Args:
        ply_file_path (str): Path to the PLY file containing the point cloud
        camera_params (dict): Camera parameters
        
    Returns:
        tuple: (point_cloud, camera_frame, view_params) for visualization
    """
    # Load the point cloud
    pcd = o3d.io.read_point_cloud(ply_file_path)
    
    # Create camera visualization
    camera_frame = create_camera_visualization(camera_params)
    
    # Calculate view parameters
    view_params = calculate_view_parameters(camera_params)
    
    return pcd, camera_frame, view_params

def main():
    # Example camera data
    camera_data = "frame000173.png,18.22329442651033,-20.86151241977417,23.03896778914268,-75.76833745708265,68.72948703510326,9.525525394693116,29.76395058435378,-2.782460227659963e-003,5.990597819531608e-003,0.1230632195011814,-0.5429382500578283,0.9354590994128915,0,0,0"
    
    # Parse camera parameters
    camera_params = parse_camera_params(camera_data)
    
    # Path to your point cloud PLY file
    ply_file = "C:/Users/nguye/Downloads/image2/test2.ply"
    
    # Render point cloud from camera viewpoint
    pcd, camera_frame, view_params = render_point_cloud_from_camera(ply_file, camera_params)
    
    # Combine geometries for visualization
    geometries = [pcd, camera_frame]
    
    # Print the calculated view parameters
    print("Calculated view parameters:")
    print(f"zoom: {view_params['zoom']}")
    print(f"front: {view_params['front']}")
    print(f"lookat: {view_params['lookat']}")
    print(f"up: {view_params['up']}")
    
    # Visualize the point cloud and camera frame using the calculated parameters
    o3d.visualization.draw_geometries(geometries,
                                     zoom=view_params['zoom'],
                                     front=view_params['front'],
                                     lookat=view_params['lookat'],
                                     up=view_params['up'])

if __name__ == "__main__":
    main()

Calculated view parameters:
zoom: 1.0952790116870754
front: [-0.06552886380179754, 0.9315077821385602, -0.3577697860694028]
lookat: [17.248095494487554, -6.998836621517622, 17.714646672570417]
up: [0.9938571426062588, 0.09297257584005073, 0.060033992305932246]


: 